In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import pandas as pd

In [3]:
# Inicializar el driver de Selenium
driver = webdriver.Chrome()

# Navegar a la página de comentarios
driver.get("https://www.atrapalo.com/entradas/comedy-nights-monologos-en-beer-station_e4857642/opiniones")

# Esperar a que se cargue la página
time.sleep(5)

# Aceptar las cookies
driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyButtonAccept").click()
time.sleep(2)

# Crear un diccionario para almacenar las fechas y notas de los comentarios
notas_beerstation = {"fecha": [], "nota_total": [], "grupo_de": [], "calidad_del_espectáculo": [], "puesta_en_escena": [],"interpretación_artística": []}

# Obtener el número total de páginas de comentarios
num_pages = int(driver.find_element(By.CLASS_NAME, "totalresultados_2").text) // 10 + 1

# Iterar sobre todas las páginas de comentarios
for page_number in tqdm(range(1, num_pages + 1)):
    # Obtener todos los comentarios de la página actual
    comentarios = driver.find_elements(By.CLASS_NAME, "global-opinions__users-row")

    # Iterar sobre todos los comentarios de la página actual
    for comentario in comentarios:
        # Obtener la fecha y la nota del comentario actual
        fecha = comentario.find_element(By.CLASS_NAME, "inline").text
        nota = comentario.find_element(By.CLASS_NAME, "rating-num").text
        grupo_de = comentario.find_element(By.CLASS_NAME, "global-opinions__users-detail-with").text
        Calidad_del_espectáculo = comentario.find_element(By.CSS_SELECTOR, ".global-opinions__users--criteria-container li:nth-child(1) .circular-progress__value").text
        Puesta_en_escena = comentario.find_element(By.CSS_SELECTOR, ".global-opinions__users--criteria-container li:nth-child(2) .circular-progress__value").text
        Interpretación_artística = comentario.find_element(By.CSS_SELECTOR, ".global-opinions__users--criteria-container li:nth-child(3) .circular-progress__value").text


        # Agregar la fecha y la nota al diccionario
        notas_beerstation["fecha"].append(fecha)
        notas_beerstation["nota_total"].append(nota)
        notas_beerstation["grupo_de"].append(grupo_de)
        notas_beerstation["calidad_del_espectáculo"].append(Calidad_del_espectáculo)
        notas_beerstation["puesta_en_escena"].append(Puesta_en_escena)
        notas_beerstation["interpretación_artística"].append(Interpretación_artística)

    # Hacer clic en el enlace de la siguiente página de comentarios
    if page_number < num_pages:
        next_page_link = driver.find_element(By.XPATH, "//a[@data-page='" + str(page_number + 1) + "']")
        next_page_link.click()
        time.sleep(120)  # Esperar a que se cargue la página

# Pasamos a Data Frame los datos
df_notas_beerstation = pd.DataFrame(notas_beerstation)

# Cerrar el driver de Selenium
driver.quit()
df_notas_beerstation

100%|███████████████████████████████████████████████████████████████████████████████| 81/81 [2:41:53<00:00, 119.92s/it]


,fecha,nota_total,grupo_de,calidad_del_espectáculo,puesta_en_escena,interpretación_artística
0,10/08/2023,10,en pareja,10,10,10
1,20/07/2023,10,en familia,10,10,10
2,13/08/2023,10,en pareja,10,10,10
3,29/07/2023,10,en pareja,10,10,10
4,16/07/2023,10,en pareja,10,10,10
...,...,...,...,...,...,...
802,16/02/2020,0.8,en pareja,2.5,0,0
803,16/02/2020,2.5,en pareja,2.5,2.5,2.5
804,15/05/2019,5.0,en pareja,5,5,5
805,15/05/2019,7.5,con amigos,7.5,7.5,7.5


In [4]:
# Especifico la ruta y el nombre del archivo CSV
ruta_archivo_csv = '../01_data/comentarios_beer.csv'

# Guardo el DataFrame en un archivo CSV
df_notas_beerstation.to_csv(ruta_archivo_csv, index=False)